In [1]:
!kaggle datasets download -d jehanbhathena/weather-dataset


  0%|          | 0.00/587M [00:00<?, ?B/s]
  0%|          | 1.00M/587M [00:00<05:40, 1.81MB/s]
  0%|          | 2.00M/587M [00:00<03:49, 2.67MB/s]
  1%|          | 3.00M/587M [00:01<03:15, 3.14MB/s]
  1%|          | 4.00M/587M [00:01<02:54, 3.49MB/s]
  1%|          | 5.00M/587M [00:01<03:02, 3.35MB/s]
  1%|          | 6.00M/587M [00:01<02:45, 3.69MB/s]
  1%|          | 7.00M/587M [00:02<02:30, 4.05MB/s]
  1%|▏         | 8.00M/587M [00:02<02:30, 4.04MB/s]
  2%|▏         | 9.00M/587M [00:02<02:21, 4.27MB/s]
  2%|▏         | 10.0M/587M [00:02<02:21, 4.27MB/s]
  2%|▏         | 11.0M/587M [00:03<02:17, 4.40MB/s]
  2%|▏         | 12.0M/587M [00:03<02:11, 4.60MB/s]
  2%|▏         | 13.0M/587M [00:03<02:15, 4.43MB/s]
  2%|▏         | 14.0M/587M [00:03<02:22, 4.23MB/s]
  3%|▎         | 15.0M/587M [00:04<02:16, 4.40MB/s]
  3%|▎         | 16.0M/587M [00:04<02:13, 4.50MB/s]
  3%|▎         | 17.0M/587M [00:04<02:10, 4.57MB/s]
  3%|▎         | 18.0M/587M [00:04<02:07, 4.66MB/s]
  3%|▎         | 19.

In [ ]:
import zipfile
import os
import shutil
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
zip_file = "C:/Users/darre/Documents/GitHub/Weather-Recognition/weather-dataset.zip"
extract_dir = "C:/Users/darre/Documents/GitHub/Weather-Recognition"

# os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [ ]:
data_dir = "C:/Users/darre/Documents/GitHub/Weather-Recognition/dataset"
new_data_dir = "C:/Users/darre/Documents/GitHub/Weather-Recognition/dataset_fix"

os.makedirs(new_data_dir, exist_ok=True)

# Create train and test folders inside the new dataset folder
train_folder = os.path.join(new_data_dir, "train")
test_folder = os.path.join(new_data_dir, "test")
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

for class_folder in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_folder)
    if os.path.isdir(class_path):
        # Split the class images into train and test sets
        images = [os.path.join(class_path, image_name) for image_name in os.listdir(class_path)]
        train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

        # Create class folders inside train and test folders
        train_class_folder = os.path.join(train_folder, class_folder)
        test_class_folder = os.path.join(test_folder, class_folder)
        os.makedirs(train_class_folder, exist_ok=True)
        os.makedirs(test_class_folder, exist_ok=True)

        # Move images to the appropriate class folders
        for image_path in train_images:
            shutil.copy(image_path, os.path.join(train_class_folder, os.path.basename(image_path)))
        for image_path in test_images:
            shutil.copy(image_path, os.path.join(test_class_folder, os.path.basename(image_path)))

In [ ]:
target_size = (224, 224,)

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(11, activation='relu')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define data generators with resizing
train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    directory='C:/Users/darre/Documents/GitHub/Weather-Recognition/dataset_fix/train',
    batch_size=32,
    target_size=(224,224,),
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    directory='C:/Users/darre/Documents/GitHub/Weather-Recognition/dataset_fix/test',
    batch_size=32,
    target_size=(224,224,),
    class_mode='categorical')

# Train the model
epochs = 10
model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    verbose=2)